<a href="https://colab.research.google.com/github/cateto/python4NLP/blob/main/transformer/Transformer_tensorflow%EB%A1%9C_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)
  
  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles
  
  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
                                 i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
                                 d_model=d_model)
    
    # 배열의 짝수 인덱스 (2i)에는 사인 함수 적용
  